In [4]:
!pip install PyMuPDF
!pip insall python-docx
!pip install pytesseract

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# converting all format into text
import os
import fitz  # PyMuPDF
import pytesseract
import docx
from bs4 import BeautifulSoup
import subprocess
import re
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        # Open the PDF file
        pdf_document = fitz.open(pdf_path)

        # Iterate through each page
        for page_number in range(len(pdf_document)):
            # Get the page
            page = pdf_document.load_page(page_number)

            # Extract text from the page
            page_text = page.get_text()

            # Append page text to the result
            text += page_text

        # Close the PDF file
        pdf_document.close()

    except Exception as e:
        print("Error:", e)

    return text


def extract_text_from_image(image_path):
    text = ""
    try:
        # Perform OCR on the image
        text = pytesseract.image_to_string(image_path)
    except Exception as e:
        print("Error:", e)

    return text

def extract_text_from_docx(docx_path):
    text = ""
    try:
        # Open the DOCX file
        doc = docx.Document(docx_path)

        # Extract text from paragraphs
        for paragraph in doc.paragraphs:
            text += paragraph.text + "\n"

    except Exception as e:
        print("Error:", e)

    return text

def extract_text_from_doc(doc_path):
    text = ""
    try:
        # Use antiword to extract text from .doc files
        process = subprocess.Popen(['antiword', doc_path], stdout=subprocess.PIPE)
        output, _ = process.communicate()
        text = output.decode('utf-8')
    except Exception as e:
        print("Error:", e)

    return text

def extract_text_from_file(file_path):
    _, file_extension = os.path.splitext(file_path)
    file_extension = file_extension.lower()

    if file_extension == '.pdf':
        return extract_text_from_pdf(file_path)
    elif file_extension == '.jpg' or file_extension == '.png':
        return extract_text_from_image(file_path)
    elif file_extension == '.docx':
        return extract_text_from_docx(file_path)
    elif file_extension == '.doc':
        return extract_text_from_doc(file_path)
    else:
        print("Unsupported file format")
        return ""

# Example usage
file_path = "/content/drive/MyDrive/resumerevaler/documents20220826-1-v01cla.pdf"
extracted_text = extract_text_from_file(file_path)
print(extracted_text)


Ashwini Kalidas Vedula
Email ID: ashwini.vedula@nyu.edu | +1 (201)-616-1347 | LinkedIn: linkedin.com/in/ashwini-vedula
Education
New York University, New York, NY
Sept 2021- May 2023
Courant Institute of Mathematical Sciences and Stern School of Business
Master of Science in Information Systems - 3.6/4
Courses: Dealing with Data, Data Science for Business : Technical, Database Management Systems, Real Time
Big Data Analytics, Cloud Computing
Technical Skills
Databases & Tools : MySQL, SSMS, MS Access, PostgreSQL, PL/SQL, MongoDB
Data Analysis & Visualization Tools : Python, R , SQL, MS Excel, Tableau, Power BI
Cloud Platforms & Big Data Technologies : AWS (Redshift, EC2, S3, EMR, SageMaker, DynamoDB Table), Azure
(Storage, IoT, Virtual Machines), GCP (BigTable, DataLab), Apache Hadoop, Hive
Experience
Clark Associates Inc
Pennsylvania, USA
Financial Systems and Data Intern
May 2022 - Aug 2022
• Extracted data from varied data sources to automate Summary Income financial reports(monthly

In [8]:
#extracting skill using regrex
def parse_resume(text_data):
    resume_data = {}

    # Extracting personal information
    personal_info_pattern = re.compile(r"([^\n]+)\n\s*([^\n]+)\s*\|\s*([^\n]+)\s*\|\s*([^\n]+)")
    personal_info_match = personal_info_pattern.search(text_data)
    if personal_info_match:
        resume_data["Name"] = personal_info_match.group(1).strip()
        resume_data["Location"] = personal_info_match.group(2).strip()
        resume_data["Email"] = personal_info_match.group(3).strip()
        resume_data["Phone"] = personal_info_match.group(4).strip()

    # Extracting education
    education_pattern = re.compile(r"Education\n([\s\S]+?)(?=Technical Skills|Skills|PROFESSIONAL EXPERIENCE|Experience|Achievements|$)", re.IGNORECASE)
    education_matches = education_pattern.findall(text_data)
    if education_matches:
        resume_data["Education"] = [match.strip() for match in education_matches]
    else:
        # If there is no other section found after education, consider education until the end of the text
        education_pattern = re.compile(r"Education\n([\s\S]+?)$", re.IGNORECASE)
        education_match = education_pattern.findall(text_data)
        if education_match:
            resume_data["Education"] = [education_match[0].strip()]

    # Extracting technical skills
    skills_pattern = re.compile(r"(Technical Skills|Skills)\n([\s\S]+?)\n(?=PROFESSIONAL EXPERIENCE|Experience|Achievements|$)", re.IGNORECASE)
    skills_match = skills_pattern.search(text_data)
    if skills_match:
        skills = skills_match.group(2).strip().split("\n")
        if "Technical Skills" in skills_match.group(1):
            resume_data["Skills"] = "\n".join(skills)
        else:
            resume_data["Additional Skills"] = "\n".join(skills)

    # Extracting experience details
    experience_pattern = re.compile(r"(Experience|PROFESSIONAL EXPERIENCE)\n([\s\S]+?)\n(?=Education|Achievements|$)", re.IGNORECASE)
    experience_matches = experience_pattern.findall(text_data)
    experience_list = []
    for match in experience_matches:
        company_info, responsibilities = match[0].strip(), match[1].strip()
        experience_list.append(f"{company_info}:\n{responsibilities}")
    resume_data["Experience"] = "\n".join(experience_list)

    # Extracting achievements
    achievements_pattern = re.compile(r"Achievements\n([\s\S]+?)(?=\n[^\n]*:|$)", re.IGNORECASE)
    achievements_matches = achievements_pattern.findall(text_data)
    resume_data["Achievements"] = ", ".join([match.strip() for match in achievements_matches])
    return resume_data

def print_resume_details(resume_data):
    for key, value in resume_data.items():
        print(f"{key}:")
        if isinstance(value, list):
            for item in value:
                print(f"  - {item}")
        else:
            print(f"  {value}")



# Assuming you have the extracted text in a variable named extracted_text
parsed_resume = parse_resume(extracted_text)
print_resume_details(parsed_resume)


Name:
  Ashwini Kalidas Vedula
Location:
  Email ID: ashwini.vedula@nyu.edu
Email:
  +1 (201)-616-1347
Phone:
  LinkedIn: linkedin.com/in/ashwini-vedula
Education:
  - New York University, New York, NY
Sept 2021- May 2023
Courant Institute of Mathematical Sciences and Stern School of Business
Master of Science in Information Systems - 3.6/4
Courses: Dealing with Data, Data Science for Business : Technical, Database Management Systems, Real Time
Big Data Analytics, Cloud Computing
Skills:
  Databases & Tools : MySQL, SSMS, MS Access, PostgreSQL, PL/SQL, MongoDB
Data Analysis & Visualization Tools : Python, R , SQL, MS Excel, Tableau, Power BI
Cloud Platforms & Big Data Technologies : AWS (Redshift, EC2, S3, EMR, SageMaker, DynamoDB Table), Azure
(Storage, IoT, Virtual Machines), GCP (BigTable, DataLab), Apache Hadoop, Hive
Experience:
  Experience:
Clark Associates Inc
Pennsylvania, USA
Financial Systems and Data Intern
May 2022 - Aug 2022
• Extracted data from varied data sources to au

In [9]:
def print_resume_details(resume_data):
    for key, value in resume_data.items():
        print(f"{key}:")
        if isinstance(value, list):
            for item in value:
                print(f"  - {item}")
        else:
            print(f"  {value}")

def extract_skills(resume_data):
    all_skills = []

    # Regular expression to match skills and sub-skills
    skill_pattern = re.compile(r"([^\n:]+):?")  # Matches any characters except newline and colon, optionally followed by a colon

    if "Skills" in resume_data:
        skills_match = skill_pattern.findall(resume_data["Skills"])
        for skill in skills_match:
            all_skills.extend([sub_skill.strip() for sub_skill in re.split(r'[;,]', skill)])

    if "Additional Skills" in resume_data:
        additional_skills_match = skill_pattern.findall(resume_data["Additional Skills"])
        for skill in additional_skills_match:
            all_skills.extend([sub_skill.strip() for sub_skill in re.split(r'[;,]', skill)])

    return all_skills


print("\nExtracted Skills:")
skills = extract_skills(parsed_resume)
for skill in skills:
    print("  -", skill)




Extracted Skills:
  - Databases & Tools
  - MySQL
  - SSMS
  - MS Access
  - PostgreSQL
  - PL/SQL
  - MongoDB
  - Data Analysis & Visualization Tools
  - Python
  - R
  - SQL
  - MS Excel
  - Tableau
  - Power BI
  - Cloud Platforms & Big Data Technologies
  - AWS (Redshift
  - EC2
  - S3
  - EMR
  - SageMaker
  - DynamoDB Table)
  - Azure
  - (Storage
  - IoT
  - Virtual Machines)
  - GCP (BigTable
  - DataLab)
  - Apache Hadoop
  - Hive


In [10]:
#skill extraction using dataset
import pandas as pd
import re

def extract_skills_from_resume(text, skills_file):
    # Load skills list from Excel file
    skills_df = pd.read_excel(skills_file)

    # Remove leading/trailing whitespace from skills
    skills_df['Category'] = skills_df['Category'].str.strip()

    # Initialize list to store matched skills
    matched_skills = set()

    # Iterate through each skill in the list
    for skill in skills_df['Category']:
        # Create a regex pattern for the skill (escaping special characters)
        pattern = r"\b{}\b".format(re.escape(skill))

        # Search for the skill in the resume text, ignoring case
        match = re.search(pattern, text, re.IGNORECASE)

        # If a match is found, add the skill to the list of matched skills
        if match:
            matched_skills.add(skill)

    return matched_skills

# Path to the Excel file containing technology skills list
skills_file_path = "/content/drive/MyDrive/resumerevaler/csv/Technology_Skills_Competencies.xlsx"

# Extract skills from the resume text based on the provided skills list
matched_skills = extract_skills_from_resume(extracted_text, skills_file_path)

# Print the matched skills
print("Matched Skills:", matched_skills)


Matched Skills: {'Database management systems', 'MongoDB', 'Software', 'Apache Hadoop', 'Python', 'PostgreSQL', 'MySQL', 'R', 'LinkedIn', 'Tableau'}


In [11]:
#occupation

import pandas as pd
import re

def extract_occupations_from_resume(text, occupations_file):
    # Load occupations list from Excel file
    occupations_df = pd.read_excel(occupations_file)

    # Remove leading/trailing whitespace from occupations
    occupations_df['Title'] = occupations_df['Title'].str.strip()

    # Initialize set to store matched occupations
    matched_occupations = set()

    # Iterate through each occupation in the list
    for occupation in occupations_df['Title']:
        # Create a regex pattern for the occupation (escaping special characters)
        pattern = r"\b{}\b".format(re.escape(occupation))

        # Search for the occupation in the resume text, ignoring case
        match = re.search(pattern, text, re.IGNORECASE)

        # If a match is found, add the occupation to the set of matched occupations
        if match:
            matched_occupations.add(occupation)

    return matched_occupations

# Path to the Excel file containing occupations list
occupations_file_path = "/content/drive/MyDrive/resumerevaler/csv/OccupationData.xlsx"

# Extract occupations from the resume text based on the provided occupations list
matched_occupations = extract_occupations_from_resume(extracted_text, occupations_file_path)

# Print the matched occupations
print("Matched Occupations:", matched_occupations)


Matched Occupations: set()
